In [ ]:
from keras.applications import VGG16
vgg16_base = VGG16(weights='imagenet',include_top=False,input_shape=(320, 480, 3))#(480, 320, 3))

from keras import models, layers 
vgg16tlp = models.Sequential()
vgg16tlp.add(vgg16_base)
vgg16tlp.add(layers.UpSampling2D(size=(2, 2)))
vgg16tlp.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
vgg16tlp.add(layers.UpSampling2D(size=(2, 2)))
vgg16tlp.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
vgg16tlp.add(layers.UpSampling2D(size=(2, 2)))
vgg16tlp.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
vgg16tlp.add(layers.UpSampling2D(size=(2, 2)))
vgg16tlp.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
vgg16tlp.add(layers.UpSampling2D(size=(2, 2)))
vgg16tlp.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
vgg16tlp.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
vgg16tlp.add(layers.Conv2D(2, (1, 1),activation='softmax'))
vgg16tlp.summary()

vgg16_base.trainable = True

In [ ]:
def f1 (y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tp = K.sum(y_true_f * y_pred_f)        
    fp = K.sum(K.clip(K.clip(y_pred_f+y_true_f,0,1)-y_true_f,0,1))
    fn = K.sum(K.clip(K.clip(y_pred_f+y_true_f,0,1)-y_pred_f,0,1))
    Pr = tp/(tp+fp)
    Re = tp/(tp+fn)
    f1t = 2*(Pr*Re)/(Pr+Re)
    return f1t

In [ ]:
import os
from glob import glob
import re
import os.path
import numpy as np
import scipy.misc
import random
import keras.backend as K
import imageio

image_shape = (480,320)
data_dir = './data_mio'
namefile = 'vgg16u'
batch_size = 5

def generator(data_folder, image_shape, batch_size):
    """  Create batches of training and validation data """
    image_paths = glob(os.path.join(data_folder, 'images', '*.png'))
    label_paths = {
        re.sub(r'_road_', '_', os.path.basename(path)): path
        for path in glob(os.path.join(data_folder, 'labels', '*_road_*.png'))}
    
    background_color = np.array([255, 0, 255])
    
    random.shuffle(image_paths)
    while 1: 
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file = label_paths[os.path.basename(image_file)]
                image = imageio.imread(image_file)
                gt_image = imageio.imread(gt_image_file)
                gt_bg = np.all(gt_image == background_color, axis=2)
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1) 
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)
                images.append(image)
                gt_images.append(gt_image)
            
            X = np.array(images)
            Y = np.array(gt_images)
            yield (X, Y)
        
train_generator = generator(os.path.join(data_dir, 'training'), image_shape, batch_size)
validation_generator = generator(os.path.join(data_dir, 'validation'), image_shape, batch_size)
test_generator = generator(os.path.join(data_dir, 'test'), image_shape, batch_size)

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras import optimizers

vgg16tlp.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5),metrics=[f1])
checkpointer = ModelCheckpoint(filepath="vgg16crack.h5", monitor='val_loss', verbose=0, 
                               save_best_only=True, mode='max', period=1)

h = vgg16tlp.fit_generator(generator = train_generator,
                           steps_per_epoch=100//batch_size,
                           epochs=300,
                           validation_data=validation_generator,
                           validation_steps=50//batch_size, 
                           callbacks=[checkpointer])  

score_train = vgg16tlp.evaluate_generator(train_generator, steps=18//batch_size)
print('training', score_train)
score_val = vgg16tlp.evaluate_generator(validation_generator, steps=18//batch_size)
print('validation', score_val)
score_test = vgg16tlp.evaluate_generator(test_generator, steps=38//batch_size)
print('test', score_test)

In [ ]:
import time
import matplotlib.pyplot as plt

image_shape = (320,480)

plt.rcParams['figure.figsize'] = (15,15)
data_folder = './resultados/cfd/' 
th_segment = 0.5
k = 10
filelist = glob(os.path.join(data_folder, '*.png'))
i = 1
inicio = time.time()
for image_file in filelist[0:k]:
    img_original = imageio.imread(image_file)
    img = scipy.misc.imresize(img_original, image_shape) 
    y = vgg16tlp.predict(np.expand_dims(img, axis=0), batch_size=1)   
    mask = np.stack([y[0,:,:,0] > th_segment,np.zeros(image_shape),np.zeros(image_shape)], axis=2)
    masked = np.ma.masked_array(mask, img).astype('float32')
    z = str(i).zfill(3)
    scipy.misc.imsave('./resultados/vgg16/CFD/'+z+'_cfd.png',masked)
    i += 1
    
fin = time.time()
print ((fin-inicio)/10)

In [ ]:
data_folder = './resultados/vgg16/cfd/'
k = 10
filelist = glob(os.path.join(data_folder, '*.png'))
i = 1
f1t = 0
prt = 0
ret = 0
for image_file in filelist[0:k]:
    img = scipy.misc.imread(image_file)
    img = np.asarray(img)
    
    #a = scipy.misc.imread('./resultados/labels_aiglern/'+str(i).zfill(3)+'b.png')  
    a = scipy.misc.imread('./resultados/labels_cfd/'+str(i).zfill(3)+'b.png')
    
    # Handed segmentation
    b = np.empty([len(a),len(a[0])])
    for m in range (len(a)):
        for n in range (len (a[0])):
            if a[m][n][0] == 0:
                b[m][n] = 0
            else:
                b[m][n] = 1
                
    c = np.empty([len(img),len(img[0])])
    for m in range (len(img)):
        for n in range (len (img[0])):
            if img[m][n][0] == 0:
                c[m][n] = 0
            else:
                c[m][n] = 1

    tp = 0
    fp = 0
    fn = 0
    for r in range(len(c)):
        for o in range(len(c[0])):
            if c[r][o] == 1 and b[r-5:r+5,o-5:o+5].sum() >= 1:
                tp += 1
            elif c[r][o] == 1 and b[r][o] == 0:
                fp += 1
            elif c[r-5:r+5,o-5:o+5].sum() == 0 and b[r][o] == 1:
                fn += 1
    pr = tp/(tp+fp+0.001)
    re = tp/(tp+fn+0.001)
    f1 = 2*(pr*re)/(pr+re+0.001)
    print (i,' Pr: ',pr, 'Re: ', re, 'F1: ', f1)
    
    f1t = (f1t+f1)
    prt = prt+pr
    ret = ret+re
    
    i += 1

print (' PrT: ', prt/10, ' ReT: ', ret/10, 'F1T: ', f1t/10)